# Chapter 2. An Array of Sequences
---
## ToC

[When a List Is Not the Answer](#when-a-list-is-not-the-answer)

1. [Arrays](#arrays)  
2. [Memory Views](#memory-views)

---

## When a List Is Not the Answer

The `list` type is flexible and easy to use, but depending on specific requirements,
there are better options. For example, an `array` saves a lot of memory when you need
to handle millions of floating-point values. On the other hand, if you are constantly
adding and removing items from opposite ends of a list, it’s good to know that a
`deque` (double-ended queue) is a more efficient FIFO data structure.

![Figure 29](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/29.PNG)

### Arrays

If a list only contains numbers, an `array.array` is a more efficient replacement. Arrays support all mutable sequence operations (including `.pop`, `.insert`, and `.extend`), as well as additional methods for fast loading and saving, such as `.frombytes` and `.tofile`. An array of float values does not hold full-fledged float instances, but only the packed bytes representing their machine values—similar to an array of double in the C language

### Summary of Key Differences

| Feature         | Tuple                        | array.array / numpy.array         |
|----------------|------------------------------|-----------------------------------|
| Type flexibility | Heterogeneous                 | Homogeneous (fixed type)          |
| Memory layout   | Pointers to PyObjects         | Raw contiguous memory             |
| Performance     | Slower for numeric ops        | Fast for numeric operations       |
| Mutability      | Immutable (tuple)             | Mutable                           |
| Use cases       | General-purpose grouping      | Efficient numeric storage/computing |

For more details on the memory layout differences, visit [Array VS Tuple](https://github.com/berserkhmdvhb/Training-Python/blob/main/src/Part_I/Chapter_2_ArrayOfSequences/ArrayVSTuple.md)

In [8]:
import sys
import array
import numpy as np

# Mixed-type tuple
t = (1, 'a', 3.14)
print("TUPLE:")
print(f"Total size of tuple: {sys.getsizeof(t)} bytes")
for i, item in enumerate(t):
    print(f"  Element {i}: value={item}, id={id(item)}, size={sys.getsizeof(item)} bytes")

TUPLE:
Total size of tuple: 64 bytes
  Element 0: value=1, id=140728964539304, size=28 bytes
  Element 1: value=a, id=140728964603824, size=42 bytes
  Element 2: value=3.14, id=2547825730000, size=24 bytes


In [9]:
# Integer array.array
a = array.array('i', [1, 2, 3])
print("\nARRAY.ARRAY:")
print(f"Total size of array.array: {sys.getsizeof(a)} bytes")
for i, item in enumerate(a):
    print(f"  Element {i}: value={item}")


ARRAY.ARRAY:
Total size of array.array: 92 bytes
  Element 0: value=1
  Element 1: value=2
  Element 2: value=3


In [4]:
# NumPy array
n = np.array([1, 2, 3], dtype=np.int32)
print("\nNUMPY.ARRAY:")
print(f"Total size of numpy.array: {n.nbytes} bytes (data only)")
print(f"Total size including metadata: {sys.getsizeof(n)} bytes")
for i, item in enumerate(n):
    address = n.ctypes.data + i * n.itemsize
    print(f"  Element {i}: value={item}, address offset={address}")


NUMPY.ARRAY:
Total size of numpy.array: 12 bytes (data only)
Total size including metadata: 124 bytes
  Element 0: value=1, address offset=2547439477264
  Element 1: value=2, address offset=2547439477268
  Element 2: value=3, address offset=2547439477272


When creating an array, you provide a typecode, a letter to determine the underlying C
type used to store each item in the array. For example, b is the typecode for what
C calls a signed char, an integer ranging from –128 to 127. If you create an
`array('b')`, then each item will be stored in a single byte and interpreted as an integer.
For large sequences of numbers, this saves a lot of memory.

More info: [Python Docs - array](https://docs.python.org/3/library/array.html)

In [29]:
array.array('b', [-128,127])

array('b', [-128, 127])

In [30]:
array.array('b', [-129,127])

OverflowError: signed char is less than minimum

In [31]:
array.array('b', [-128,128])

OverflowError: signed char is greater than maximum

**Creating, saving, and loading a large array of floats**

In [2]:
from array import array
from random import random
floats = array('d', (random() for i in range(10**7)))
floats[-1]

0.5041742820295562

In [ ]:
fp = open('floats.bin', 'wb')
# Save the array to a binary file.
floats.tofile(fp)
fp.close()

In [ ]:
# Create an empty array of doubles.
floats2 = array('d')
fp = open('floats.bin', 'rb')
# Read 10 million numbers from the binary file.
floats2.fromfile(fp, 10**7)
fp.close()
floats2[-1]

0.5041742820295562

In [ ]:
# Verify that the contents of the arrays match.
floats2 == floats

True

![Figure 30](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/30.PNG)

![Figure 31](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/31.PNG)

## Memory Views

The built-in memoryview class is a shared-memory sequence type that lets you handle slices of arrays without copying bytes. It was inspired by the NumPy library. Travis Oliphant, lead author of NumPy, answers the question, ["When should a memoryview be used?"](https://fpy.li/2-17) like this:

> A memoryview is essentially a generalized NumPy array structure in Python itself
(without the math). It allows you to share memory between data-structures (things like
PIL images, SQLite databases, NumPy arrays, etc.) without first copying. This is very
important for large data sets.

Using notation similar to the `array` module, the `memoryview.cast` method lets you change the way multiple bytes are read or written as units without moving bits around. `memoryview.cast` returns yet another `memoryview` object, always sharing the same memory.

**Example:** How to create alternate views on the same array of 6 bytes, to
operate on it as a 2×3 matrix or a 3×2 matrix.

In [ ]:
from array import array
# Build array of 6 bytes (typecode 'B').
octets = array('B', range(6))
octets

array('B', [0, 1, 2, 3, 4, 5])

In [ ]:
# Build memoryview from that array, then export it as a list.
m1 = memoryview(octets)
m1.tolist()

[0, 1, 2, 3, 4, 5]

In [11]:
# Build new memoryview from that previous one, but with 2 rows and 3 columns
m2 = m1.cast('B', [2, 3])
m2.tolist()

[[0, 1, 2], [3, 4, 5]]

In [12]:
# 3 rows 2 columns
m3 = m1.cast('B', [3, 2])
m3.tolist()

[[0, 1], [2, 3], [4, 5]]

In [14]:
# proving that the memory was shared among octets, m1, m2, and m3.
m2[1, 1] = 22
m3[1, 1] = 33
octets

array('B', [0, 1, 2, 33, 22, 5])

**Example:** How to to change a single byte of an item in an array of 16-bit integers.

In [53]:
numbers = array('h', [-2, -1, 0, 1, 2])
# Build memoryview from array of 5 16-bit signed integers (typecode 'h').
memv = memoryview(numbers)
memv.tolist()

[-2, -1, 0, 1, 2]

In [54]:
memv[0]

-2

In [55]:
# Create memv_oct by casting the elements of memv to bytes (typecode 'B').
memv_oct = memv.cast('B')
# Export elements of memv_oct as a list of 10 bytes, for inspection.
memv_oct.tolist()

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

Explanation of values transformed:

| Int Value | 16-bit Binary        | Bytes (little-endian)  | Interpreted Unsigned Bytes  |
|-----------|----------------------|------------------------|-----------------------------|
|    -2     | 11111111 11111110    | 0xFE 0xFF              | 254, 255                    |
|    -1     | 11111111 11111111    | 0xFF 0xFF              | 255, 255                    |
|     0     | 00000000 00000000    | 0x00 0x00              | 0, 0                        |
|     1     | 00000000 00000001    | 0x01 0x00              | 1, 0                        |
|     2     | 00000000 00000010    | 0x02 0x00              | 2, 0                        |
|     4     | 00000000 00000100    | 0x04 0x00              | 4, 0                        |
|  1024     | 00000100 00000000    | 0x00 0x04              | 0, 4                        |

In [51]:
# a 4 in the most significant byte of a 2-byte unsigned integer is 1024.
memv_oct[5] = 4
memv_oct.tolist()

[254, 255, 255, 255, 0, 4, 1, 0, 2, 0]

In [52]:
numbers

array('h', [-2, -1, 1024, 1, 2])

```
[0,4]

First byte = 0 → 00000000 (low byte)

Second byte = 4 → 00000100 (high byte)

high byte + low byte = 00000100 00000000

   4           0     
00000100   00000000  
   ↑           ↑     
high byte   low byte 
```
$$2^{10} = 1024$$

![Figure 32](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/32.PNG)

Link: [fluentpython.com: “Parsing binary records with struct”.](https://fpy.li/2-18)

## NumPy